# Example: reading/writing flat ROOT ntuples with treestream. 
 >__Created__:  24-Sep-2018 Harrison B. Prosper
 
 
Writing code to create and and read flat __ROOT__ ntuples (ones that store the standard types, __int__, __long__, __float__, __double__, __string__ and arrays thereof) is a routine task. It is so routine in fact that this chore ought to be done for you...by a machine. That is the purpose of the package __treestream__. It handles all the boilerplate __ROOT__ code for creating and reading flat __ROOT__ ntuples and does so through a very simple interface. __treestream__ is the modern incarnation of a package written in 2001 when the author finally got fed up of writing the same boilerplate code to create and read flat ntuples.

The package contains two classes __itreestream__ and __otreestream__ that can be called from C++ or __Python__. This example shows how to  use them.

In [1]:
import os, sys
import ROOT as rt
from ctypes import c_int, c_double
from treestream import itreestream, otreestream
# turn off ROOT use of javascript
%jsroot off

Welcome to JupyROOT 6.14/04
loaded treestream


### Write a flat ROOT ntuple

In [2]:
def write_ntuple(filename="test.root", treename="Events"):
    from time import ctime
    print "write: %s" % filename
    
    stream = otreestream(filename, treename, ctime())

    # declare variables to be written to ntuple
    # warning:
    # since Python is a dynamically typed language, writing
    # njet = 42
    # will change the type of njet! Instead:
    # 1. use the value attribute of the ctypes to get and 
    #    set their values
    # 2. use the STL vector methods in the usual way
    # 3. use the assign method to assign to the STL string
    ht      = c_double()
    njet    = c_int()
    jetet   = rt.vector("double")(20)
    comment = rt.string(80*' ')
    
    # -------------------------------------------------
    # make variables known to output tree "Events" 
    # by creating a branch for each variable
    # -------------------------------------------------
    # create branch HT of type double
    stream.add("HT", ht)
    
    # create branches njet of type int and jetEt of type
    # double. together these define a ROOT variable 
    # length array, where njet is the counter variable of
    # the array and jetEt is the variable length structure
    # with the data, which treestream assumes is modeled
    # with an STL vector. 
    stream.add("njet", njet)
    stream.add("jetEt[njet]", jetet)
    
    # create branch comment of type string
    stream.add("comment", comment)

    # loop over data to be written out
    rand    = rt.TRandom3()
    entries = 1000
    step    = 200
    for entry in range(entries):

        # generate some data
        njet.value = rand.Integer(10)
        jetet.clear()
        ht.value = 0.0 # see ctypes documentation
        for i in range(njet.value):
            jetet.push_back(rand.Exp(10))
            ht.value += jetet[i]
        comment.assign("event: %5d njet = %2d" % \
                       (entry + 1, njet.value))

        # commit data to root file
        stream.commit()

        if entry % step == 0:
            print "%5d%5d%10.2f (%s)" % (entry, 
                                         jetet.size(), 
                                         ht.value, 
                                         comment)
        
    stream.close()

### Read a flat ROOT ntuple

In [3]:
def read_ntuple(filename="test.root", treename="Events"):
    print "read: %s" % filename
    stream = itreestream(filename, treename)
    stream.ls()

    # declare variables to receive data
    # note variable length arrays are mapped to STL vectors and
    # note that here the type differs (float) from that stored
    # in the ntuple (double)
    HT      = c_double()
    ET      = rt.vector("float")(20)
    comment = rt.string(80*' ')

    # select which variables (branches) to 
    # read and where to copy their data.
    stream.select("jetEt",   ET)
    stream.select("comment", comment)
    stream.select("HT",      HT)
    
    entries = stream.entries()
    step    = 200
    for entry in range(entries):
        # read event into memory
        stream.read(entry)
        if entry % step == 0:
            print "%5d%5d%10.2f (%s)" % (entry, 
                                         ET.size(), 
                                         HT.value, 
                                         comment)       
    stream.close()    

In [4]:
def main():
    write_ntuple()
    print
    read_ntuple()

In [5]:
main()

write: test.root
    0    9     65.75 (event:     1 njet =  9)
  200    1     23.90 (event:   201 njet =  1)
  400    4     45.96 (event:   401 njet =  4)
  600    2     50.98 (event:   601 njet =  2)
  800    9     98.88 (event:   801 njet =  9)

read: test.root
    0    9     65.75 (event:     1 njet =  9)
  200    1     23.90 (event:   201 njet =  1)
  400    4     45.96 (event:   401 njet =  4)
  600    2     50.98 (event:   601 njet =  2)
  800    9     98.88 (event:   801 njet =  9)
File test.root
Tree   Events
Entries  1000

    1 Events/HT 	/ Double_t
    2 Events/comment 	/ Char_t [23]
    3 Events/jetEt 	/ Double_t 	(9) / njet
    4 Events/njet 	/ Int_t *
